In [20]:
from datetime import datetime
import pandas as pd
import numpy as np

In [21]:
dtypes = {"Date": str}
parse_dates = list(dtypes.keys())

In [22]:
def preprocess_game_columns(df):
    df["Date"] = df["Date"] + " " +  df["Time"]
    df.drop(columns=["Time", "LTime"], inplace=True)
    df["Date"] = pd.to_datetime(df["Date"])

    return df

In [51]:
def get_individual_data_files(input_dir):
    # first third of our data files
    opp_first_downs = pd.read_csv(input_dir+"opp_first_downs.csv").drop(columns=["Rk", "OT"]).rename(columns={"Unnamed: 6": "Home"})
    tm_first_downs = pd.read_csv(input_dir+"tm_first_downs.csv").drop(columns=["Rk", "OT"]).rename(columns={"Unnamed: 6": "Home"})
    penalties = pd.read_csv(input_dir+"penalties.csv").drop(columns=["Rk", "OT"]).rename(columns={"Unnamed: 6": "Home"})

       # second third of our data files
    tm_pass_comp = pd.read_csv(input_dir+"tm_passing_comp.csv").drop(columns=["Rk", "OT"]).rename(columns={"Unnamed: 6": "Home"})
    tm_rush_yds = pd.read_csv(input_dir+"tm_rushing_yards.csv").drop(columns=["Rk", "OT"]).rename(columns={"Unnamed: 6": "Home"})
    tm_tot_yds = pd.read_csv(input_dir+"tm_total_yards.csv").drop(columns=["Rk", "OT"]).rename(columns={"Unnamed: 6": "Home"})

    # final third of our data files
    opp_pass_comp = pd.read_csv(input_dir+"opp_pass_comp.csv").drop(columns=["Rk", "OT"]).rename(columns={"Unnamed: 6": "Home"})
    opp_rush_yds = pd.read_csv(input_dir+"opp_rush_yds.csv").drop(columns=["Rk", "OT"]).rename(columns={"Unnamed: 6": "Home"})
    opp_tot_yds = pd.read_csv(input_dir+"opp_total_yds.csv").drop(columns=["Rk", "OT"]).rename(columns={"Unnamed: 6": "Home"})
    punts_temperature = pd.read_csv(input_dir+"punts_temperature.csv").drop(columns=["Rk", "OT"]).rename(columns={"Unnamed: 6": "Home"})

    tm_first_downs = preprocess_game_columns(tm_first_downs)
    opp_first_downs = preprocess_game_columns(opp_first_downs)

    tm_pass_comp = preprocess_game_columns(tm_pass_comp)
    opp_pass_comp = preprocess_game_columns(opp_pass_comp)

    tm_rush_yds = preprocess_game_columns(tm_rush_yds)
    opp_rush_yds = preprocess_game_columns(opp_rush_yds)

    tm_tot_yds = preprocess_game_columns(tm_tot_yds)
    opp_tot_yds = preprocess_game_columns(opp_tot_yds)

    penalties = preprocess_game_columns(penalties)
    punts_temperature = preprocess_game_columns(punts_temperature)

    game_columns = ['Tm', 'Year', 'Date', 'Home', 'Opp', 'Week', 'G#', 'Day', 'Result']
    tm_first_downs.columns = game_columns + ['Tm_1stD', 'Tm_Rsh1stD', 'Tm_Pass1stD', 'Tm_Pen1stD', 'Tm_3DAtt', 'Tm_3DConv', 'Tm_3D%', 'Tm_4DAtt', 'Tm_4DConv', 'Tm_4D%']
    opp_first_downs.columns = game_columns + ['Opp_1stD', 'Opp_Rush1stD', 'Opp_Pass1stD', 'Opp_Pen1stD']
    penalties.columns = game_columns + ["Tm_Pen", "Tm_Yds", "Opp_Pen", "Opp_Yds", "Comb_Pen", "Comb_Yds"]

    opp_pass_comp.columns = game_columns + ["Opp_PassCmp","Opp_PassAtt","Opp_PassCmp%","Opp_PassYds","Opp_PassTD","Opp_Int","Opp_Sk","Opp_SkYds","Opp_QBRating"]
    opp_rush_yds.columns = game_columns + ["Opp_RshAtt","Opp_RshYds","Opp_RshY/A","Opp_RshTD"]
    punts_temperature.columns =game_columns+ ["Tm_Pnt","Tm_PntYds","Tm_Y/P","Tm_Surface","Tm_Roof","Tm_Temperature"]

    tm_rush_yds.columns = game_columns + ["Tm_RshAtt", "Tm_RshYds", "Tm_RshY/A", "Tm_RshTD"]
    tm_pass_comp.columns = game_columns + ["Tm_cmp","Tm_PassAtt", "Tm_PassCmp%", "Tm_PassYds", "Tm_PassTD", "Tm_INT","Tm_Sk","Tm_SkYds","Tm_QBRating"]
    tm_tot_yds.columns = game_columns + ["Tm_TotYds","Tm_Plys","Tm_Y/P","Tm_DPlys","Tm_DY/P", "Tm_TO","Tm_TOP","Tm_Gametime"]

    first_third = tm_first_downs.merge(penalties, how="left", on=game_columns).merge(opp_first_downs, how="left", on=game_columns)
    second_third = opp_pass_comp.merge(opp_rush_yds, how="left", on=game_columns).merge(punts_temperature, how="left", on=game_columns)
    third_third = tm_rush_yds.merge(tm_pass_comp, how="left", on=game_columns).merge(tm_tot_yds, how="left", on=game_columns)
    final_data= first_third.merge(second_third, how="left", on=game_columns).merge(third_third, how="left", on=game_columns)


    return final_data

In [52]:
input1= 'C:/Users/ly264/NFL-Analysis/src/data/'

In [53]:
result1 = get_individual_data_files(input1)

In [56]:
result1

,Tm,Year,Date,Home,Opp,Week,G#,Day,Result,Tm_1stD,...,Tm_SkYds,Tm_QBRating,Tm_TotYds,Tm_Plys,Tm_Y/P_y,Tm_DPlys,Tm_DY/P,Tm_TO,Tm_TOP,Tm_Gametime
0,NOR,2013,2013-11-10 08:30:00,NaN,DAL,10,9,Sun,W 49-17,40,...,9.0,139.0,625.0,80.0,7.8,43.0,4.5,1.0,39:32:00,3:13
1,DEN,2013,2013-12-08 04:05:00,NaN,TEN,14,13,Sun,W 51-28,39,...,0.0,107.8,551.0,91.0,6.1,48.0,5.3,NaN,39:20:00,3:17
2,HOU,2012,2012-11-18 01:03:00,NaN,JAX,11,10,Sun,W 43-37,39,...,23.0,121.7,653.0,92.0,7.1,66.0,6.9,3.0,41:40:00,3:49
3,DET,2012,2012-09-23 01:03:00,@,TEN,3,3,Sun,L 41-44,36,...,8.0,118.9,583.0,92.0,6.3,61.0,7.2,1.0,40:36:00,3:51
4,KAN,2004,2004-10-24 01:00:00,NaN,ATL,7,6,Sun,W 56-10,36,...,0.0,101.6,540.0,77.0,7.0,50.0,4.4,1.0,38:54:00,3:05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11460,NYJ,2021,2022-01-09 01:00:00,@,BUF,18,17,Sun,L 10-27,4,...,82.0,66.0,53.0,46.0,1.2,79.0,5.4,NaN,23:00,3:11
11461,HOU,2002,2002-12-08 01:01:00,@,PIT,14,13,Sun,W 24-6,3,...,23.0,40.8,47.0,40.0,1.2,95.0,4.4,1.0,20:19,3:09
11462,MIN,2006,2006-12-21 08:05:00,@,GNB,16,15,Thu,L 7-9,3,...,23.0,35.4,104.0,45.0,2.3,76.0,4.2,1.0,25:21:00,3:08
11463,OAK,2008,2008-11-02 04:15:00,NaN,ATL,9,8,Sun,L 0-24,3,...,21.0,19.0,77.0,34.0,2.3,82.0,5.5,2.0,14:45,2:40
